In [1]:
import cv2
import numpy as np

In [11]:
img = cv2.imread("C:/Users/sumit/OneDrive/Desktop/cvimg/sudoku.png")
img = cv2.resize(img,(400,400))

gry = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)          #Hough Line Transform works best on single-channel images.
edg = cv2.Canny(gry,20,250)                         #Uses Canny Edge Detector.,Thresholds are 20 (low) and 250 (high).

lines = cv2.HoughLines(edg,1,np.pi/180,200)         #1 → resolution of rho (distance from origin to line) = 1 pixel.,np.pi/180 → resolution of theta (angle step) = 1 degree.

for r,th in lines[0]:
    a = np.cos(th)
    b = np.sin(th)

    x0 = a*r
    y0 = b*r

    x1 = int(x0+1000*(-b))                      #1000? → It’s just a large number, ensures the line extends outside the image size.
    y1 = int(x0+1000*(a))
    x2 = int(x0-1000*(-b))
    y2 = int(x0-1000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)

cv2.imshow("org",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# shortest method

In [12]:
img = cv2.imread("C:/Users/sumit/OneDrive/Desktop/cvimg/sudoku.png")
img = cv2.resize(img,(400,400))

gry = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)          
edg = cv2.Canny(gry,20,250)

l = cv2.HoughLinesP(edg,1,np.pi/180,200,minLineLength = 180,maxLineGap = 100)

for i in l:
    x1,y1,x2,y2 = i[0]
    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)

cv2.imshow("org",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Template Matching 